In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import torch
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [7]:
from drive.MyDrive.how_good_is_my_plot.fine_tune_for_plots import train_model, initialize_model
from drive.MyDrive.how_good_is_my_plot.regression_dataset import RegressionDataset

from __future__ import print_function
from __future__ import division

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import time
import os
import copy
import numpy as np

# The split we are training and testing on
split = 1

# Data directory on which you want to train the model
data_dir = f"drive/MyDrive/how_good_is_my_plot/final_plots_forMT"

np.random.seed = 0

# Number of classes in the dataset
num_classes = 1

# Batch size for training (change depending on how much memory you have)
batch_size = 10

# Number of epochs to train for  -> can leave this on 50 - the model with best validation accuracy will be saved.
num_epochs = 50

# Initialize the model for this run
input_size = 224
model_ft = initialize_model(num_classes, use_pretrained=True)

# Resizing and normalizing data
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: RegressionDataset(data_dir, f"drive/MyDrive/how_good_is_my_plot/csv_splits/split{split}_{x}.csv",
                                        data_transforms[x]) for x in ['train', 'val']}

# Create training and validation dataloaders TUKROJREOGJOREJGRGJEOŠREJOREJGŠER
dataloaders_dict = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in
    ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Send the model to GPU
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()

# you can change learning rate here
learning_rate = 0.0001
optimizer_ft = optim.Adam(params_to_update, lr=learning_rate)

# Setup the loss fxn
criterion = nn.MSELoss()

# Train and evaluate
model_ft = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, device, num_epochs=num_epochs, regression=True)

torch.save(model_ft.state_dict(), f"drive/MyDrive/how_good_is_my_plot/cnn_weights/resnet101_quality.pth")

Initializing Datasets and Dataloaders...
Epoch 0/49
----------
train Loss: 159.2355
val Loss: 8.8098

Epoch 1/49
----------
train Loss: 143.4390
val Loss: 36.4210

Epoch 2/49
----------


KeyboardInterrupt: ignored